In [1]:
# Pacotes utilizados
using PyPlot
using LinearAlgebra
using Random
using Statistics
using Printf
using JLD2, FileIO
using ProgressMeter
include("Ilearning.jl")
rng = MersenneTwister(1234);

In [8]:
# Número de memorias
Pv = 1:5:40
N = 200
# Number of simulations
Nsim = 300

# numero maximo de iterações
it_max = 1000

1000

In [9]:
NoiseValues = 0.3

0.3

In [10]:
BinaryStates, ComplexStates = ILearning.Multi_Estados()

([1, -1], Complex{Float64}[1.0 + 1.0im, 1.0 - 1.0im, -1.0 + 1.0im, -1.0 - 1.0im])

In [11]:
Success_total_0 = zeros(Float64,(length(Pv),));
Success_total_1 = zeros(Float64,(length(Pv),));
Success_total_2 = zeros(Float64,(length(Pv),));
Success_total_3 = zeros(Float64,(length(Pv),));

In [ ]:
@showprogress for i=1:length(Pv)
    
    P = Pv[i]
    
    println("Iteracion")
    println(i)
    println(" ")
    
    Success_0 = zeros(Int64,(Nsim,));
    Success_1 = zeros(Int64,(Nsim,));
    Success_2 = zeros(Int64,(Nsim,));
    Success_3 = zeros(Int64,(Nsim,)); 
    
    # Synthetic fundamental memory matrix
    U = 2*rand(rng,Bool,(N,P)).-1;
    
    for j =1:Nsim
        
        # Generic Binary Corrupted Vector
        xr = ILearning.noise(U[:,1], BinaryStates,NoiseValues)    
        
########################################################################################
### Storing Rules ######################################################################
########################################################################################        
        W1 = ILearning.train(ILearning.first,U,nothing)
        W2 = ILearning.train(ILearning.second,U,nothing)
        Wc = ILearning.Correlation(U,nothing);
        Wp = ILearning.Projection(U)
      
########################################################################################
### Correlação ###################################################################
########################################################################################        
        y1 = ILearning.Asy(Wc,xr,it_max);

        Error = norm(U[:,1]-y1)

        if Error<1.e-4
            Success_0[j] = Success_0[j] + 1
        end
    
########################################################################################
### Storkey de Primeira ordem ##################################################################
########################################################################################   
        
        y2 = ILearning.Asy(W1,xr,it_max); 
        
        Error = norm(U[:,1]-y2)

        if Error<1.e-4
            Success_1[j] = Success_1[j] + 1
        end
        
        y3 = ILearning.Asy(Wp,xr,it_max); 
        
        Error = norm(U[:,1]-y3)

        if Error<1.e-4
            Success_2[j] = Success_2[j] + 1
        end 
        
        y4 = ILearning.Asy(W2,xr,it_max); 
        
        Error = norm(U[:,1]-y4)

        if Error<1.e-4
            Success_3[j] = Success_3[j] + 1
        end       
        
    Success_total_0[i] = sum(Success_0)/Nsim
    Success_total_1[i] = sum(Success_1)/Nsim
    Success_total_2[i] = sum(Success_2)/Nsim
    Success_total_3[i] = sum(Success_3)/Nsim
        
end
    
end

@save "JLDs/FixedNoise_Comparison_P.jld2" Success_total_0 Success_total_1 Success_total_2 Success_total_3  NoiseValues Pv

Iteracion
1
 


Progress:  12%|█████▏                                   |  ETA: 0:01:03

In [ ]:
@load "JLDs/FixedNoise_Comparison_P.jld2"
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
font0 = Dict(
        "font.size" => 18,
        "axes.labelweight" => "bold",
        "axes.labelsize" => 16,
        "xtick.labelsize" => 16,
        "ytick.labelsize" => 16,
        "legend.fontsize" => 13,
)
merge!(rcParams, font0)
plot(Pv,Success_total_1,linewidth=1.5,color="red",linestyle ="dashed", marker="o",markersize = 10,
             markeredgewidth=1.5, markeredgecolor=(0,0,0),label = "1st Order Storkey")

plot(Pv,Success_total_3,color="#2ca02c",linestyle ="dashed", marker=">",markersize = 10,markeredgewidth=1.5, markeredgecolor=(0,0,0),label = "2nd Order Storkey")

plot(Pv,Success_total_0,color="orange",linestyle ="solid", marker="s",markersize = 10,markeredgewidth=1.5, markeredgecolor=(0,0,0),label = "Hebb learning")

plot(Pv,Success_total_2,linewidth=1,color="blue",linestyle ="solid",markersize = 10,markeredgewidth=1.5, markeredgecolor=(0,0,0), marker="d",label = "Projection rule")

xlabel("Number of Memories")
ylabel("Recall Probability")

legend()
grid()

savefig("Figures/RealRecall_P.eps")
savefig("Figures/RealRecall_P.pdf")